In [1]:
###### Giada Barzaghi - SEPTEMBER 2021
### Modeling aviation networks: hybrid networks 

### Goal 1. integrating the concept of itinerary 
### Goal 2. insert some more realistic values for the variables in the model (eg. from Seymour, EMEP/EEA spreadsheet)
### Goal 3: REPRESENT NETWORK GRAPHICALLY + other graphs + Adobe Illustrator

# SETTING UP WORKING ENVIRONMENT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyomo.environ import *
import string

In [2]:
# CREATING SETS AND ASSIGNING VALUES TO VARIABLES

# SET OF AIRPORTS

airports = list(string.ascii_uppercase)
print(airports)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [3]:
# SET OF AIRPORT PAIRS
# ---> note: remember difference city-pair / airport-pair
AP = [(n1, n2) 
    for n1 in airports 
    for n2 in airports
     if n1!=n2]

#print(A)

# SET OF NON-STOP ITINERARIES
I0 = []
for x in AP:
    x= list(x)
    x.insert(1, "-")
    x= tuple(x)
    I0.append(x)

print(I0[1:10])

[('A', '-', 'C'), ('A', '-', 'D'), ('A', '-', 'E'), ('A', '-', 'F'), ('A', '-', 'G'), ('A', '-', 'H'), ('A', '-', 'I'), ('A', '-', 'J'), ('A', '-', 'K')]


In [4]:
# SET OF ONE-STOP ITINERARIES 
I1 = [(n1, n2, n3)
    for n1 in airports
    for n2 in airports
    for n3 in airports
    if n1!=n2
    if n1!=n3
    if n3!=n2]
print(I1[1:10])

[('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'B', 'F'), ('A', 'B', 'G'), ('A', 'B', 'H'), ('A', 'B', 'I'), ('A', 'B', 'J'), ('A', 'B', 'K'), ('A', 'B', 'L')]


In [5]:
# COMPLETE SET OF ITINERARIES 
I = I0+I1
print(I[1:10], I[-10:-1])

[('A', '-', 'C'), ('A', '-', 'D'), ('A', '-', 'E'), ('A', '-', 'F'), ('A', '-', 'G'), ('A', '-', 'H'), ('A', '-', 'I'), ('A', '-', 'J'), ('A', '-', 'K')] [('Z', 'Y', 'O'), ('Z', 'Y', 'P'), ('Z', 'Y', 'Q'), ('Z', 'Y', 'R'), ('Z', 'Y', 'S'), ('Z', 'Y', 'T'), ('Z', 'Y', 'U'), ('Z', 'Y', 'V'), ('Z', 'Y', 'W')]


In [6]:
# IMPORTING DATA FROM EEA/EMEP AIR POLLUTANT EMISSION INVENTORY GUIDEBOOK 2019
# (saved in my GitHub directory)
url = 'https://raw.githubusercontent.com/GiadaBarzaghi/MSc-Dissertation-/main/EEA%3AEMEP%20air%20pollutant%20emission%20inventory%202019.csv'
df = pd.read_csv(url,index_col=0)
df.head()

# Subsetting the imported dataframe
df = df[['Sector', 'Type', 'Technology', 'Pollutant', 'Value', 'Unit']]
# Dropping null values
df = df.dropna()

# Creating list of aircraft models 
A1 = df[['Technology']].to_numpy()
print('Number of aircraft models from EEA/EMEP: '+str(len(A1)))


####################################################################

# IMPORTING DATA FROM SEYMOUR ET AL. 2020 - FEAT MODEL 
url1 = 'https://raw.githubusercontent.com/kwdseymour/FEAT/master/aircraft_type_designators.csv'
aircraft_type_designators = pd.read_csv(url1, index_col=0)
A = aircraft_type_designators[['ac_code_icao']].to_numpy()
A = [tuple(n) for n in A]
print('Number of aircraft models from Seymour et al.2020: '+str(len(A)))

Number of aircraft models from EEA/EMEP: 464
Number of aircraft models from Seymour et al.2020: 133


In [7]:
df.info()
aircraft_type_designators.info()

<class 'pandas.core.frame.DataFrame'>
Index: 464 entries, 1.A.3.a.i.(i) to 1.A.3.a.ii.(i)
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sector      464 non-null    object 
 1   Type        464 non-null    object 
 2   Technology  464 non-null    object 
 3   Pollutant   464 non-null    object 
 4   Value       464 non-null    float64
 5   Unit        464 non-null    object 
dtypes: float64(1), object(5)
memory usage: 25.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 0 to 132
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ac_code_iata  133 non-null    object
 1   ac_code_icao  133 non-null    object
 2   bada_model    133 non-null    object
 3   ac_name_oag   133 non-null    object
 4   ac_name_ps    97 non-null     object
dtypes: object(5)
memory usage: 6.2+ KB


In [8]:
# Assigning a random capacity to each aircraft model 
# TO DO : Look up realistic values
np.random.seed(12345)
K = {}
for a in A:
    K[a]=np.random.randint(100,300)
    
#print(K)

In [9]:
# Assigning random demand to each itinerary
# -> from normal distribution ----> THEN WITH MULTINOMIAL LOGIT (?)
np.random.seed(12345)
D={}
for i in I:
    D[i]=np.random.normal(100, 10)/100
    
#print(D)

In [11]:
############## ! ! ! ! ! !
# work in progress - see Aug2021 for attemps to database creation 
# THIS IS HOW YOU ASSIGNED VALUES FOR POLLUTION IN THE P2P MODEL - temporary solution 
P = {}
for i in I:
    for a in A:
        P[i, a]=np.random.randint(50, 1000)
print(list(P.items())[0:2])
len(P)

[((('A', '-', 'B'), ('A140',)), 974), ((('A', '-', 'B'), ('A148',)), 184)]


2161250

In [12]:
# Creating a dummy for whether a flight-leg can be executed or not with a specific aircraft model:
# (0.5 probability of being yes/1)
R = {}
for i in I:
    for a in A:
        if np.random.randn()<=0.5:
            R[i, a]=1
        else:
            R[i, a]=0

In [13]:
# Assigning random values for max utilization of each aircraft type
# TO DO: LOOK UP REALISTIC ONES 
W = {}
for a in A:
    W[a] = np.random.randint(10, 50)

In [20]:
# SETTING UP THE CONCRETE MODEL 
# initializing paramaters and sets
m = ConcreteModel()
m.I = Set(initialize=I) 
m.A = Set(initialize=A)
m.K = Param(m.A, initialize=K)
m.W = Param(m.A, initialize = W)
m.D = Param(m.I, initialize=D)
m.P = Param(m.I, m.A, initialize=P)
m.R = Param(m.I, m.A, initialize=R)

# setting up decision variables
m.f=Var(m.I, m.A, domain=NonNegativeIntegers) # f = frequency of flights operated by aircraft type A on itinerary I 
m.z=Var(m.A, domain=NonNegativeIntegers) # z= number of aircrafts type A

# setting up model objective and constraints
# OBJECTIVE
def minimize_pollution(m):
    return sum(m.f[i,a]*m.P[i,a]
              for i in m.I for a in m.A)
m.objective=Objective(rule=minimize_pollution, sense=minimize)

# CONSTRAINTS 
def utilization_rule(m, a):
    return sum(m.f[i,a]
              for i in m.I)<=m.W[a]*m.z[a]
m.utilization= Constraint(m.A, rule=utilization_rule)

def demand_rule(m, i1, i2, i3):
    return sum(m.f[i1, i2, i3, a]*m.K[a]
              for a in m.A)>=m.D[i1, i2, i3]
m.demand_rule = Constraint(m.I, rule = demand_rule)

def operational_rule(m, i1, i2, i3):
    return sum(m.f[i,a] 
              for I in m.I)<=m.R[i,a]*m.f[i,a]
m.operational_rule = Constraint(m.I, rule=operational_rule)


In [ ]:
# calling the solver -> using cbc

#%%time # what do we need this for?
opt = SolverFactory('cbc', executable=r'/usr/local/bin/cbc')
opt.options['mipgap']=0.20 
results = opt.solve(m, tee=False) # tee=True to see the output of the model

In [ ]:
# PRINTING SOME RESULTS
# 1. number of aircrafts for each model type
for a in m.A:
    print('Aircraft type: '+str(a))
    print('Number of aircrafts: '+str(m.z[a].value))

# 2. frequancy of flights on itinerary i operated with aircraft type a
for i in m.I:
    for a in m.A:
        print('Flight leg: %s; \naircraft type: %s; \nnumber of flights operated: %s\n' 
             % (i, a, m.f[i,a].value))